In [1]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import numpy as np
import pandas as pd


In [2]:
df = pd.read_pickle(r'\\popnasal07.bnhpgroup.com\Departments\DIGITALFACTORY\GENERAL\קורס DS - AIRBNB\f_e.pkl')
df

,Listing ID,Host Response Rate,Accomodates,Price,Guests Included,Min Nights,Reviews,Overall Rating,review_mnt,Host_tenure,...,Communication Rating_encoded,Neighborhood Group_encoded,Location Rating_encoded,Value Rating_encoded,Bathrooms_encoded,Bedrooms_encoded,Beds_encoded,Host Response Time_encoded,Property Type_encoded,Room Type_encoded
0,2695,0.5,2.0,17.0,1.0,2.0,42.0,100.0,8.5,10.288843,...,0,6,2,0,0,2,1,2,1,1
1,3176,0.5,4.0,90.0,2.0,62.0,144.0,92.0,10.0,10.198494,...,2,6,0,2,0,2,4,2,1,0
2,7071,1.0,2.0,33.0,1.0,2.0,10076.0,96.0,7.659091,9.626283,...,0,6,0,0,0,2,4,4,1,1
3,9991,1.0,7.0,180.0,5.0,6.0,6.0,100.0,7.0,9.349760,...,0,6,0,0,1,8,9,2,1,0
4,14325,1.0,1.0,70.0,1.0,90.0,23.0,93.0,11.0,9.117043,...,0,6,2,2,0,0,1,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12212,31144871,1.0,2.0,25.0,1.0,2.0,15.0,90.0,12.0,2.255989,...,0,4,2,2,0,2,1,4,12,1
12213,31161939,0.9,2.0,25.0,1.0,2.0,14.0,90.0,12.0,0.008214,...,2,5,2,2,0,2,4,3,1,1
12214,31171455,1.0,4.0,110.0,2.0,1.0,40.0,88.0,12.0,3.173169,...,2,4,0,2,0,2,4,4,1,0
12215,31185236,1.0,1.0,40.0,1.0,1.0,10.0,100.0,12.0,0.876112,...,0,6,0,0,0,2,1,3,3,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12217 entries, 0 to 12216
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Listing ID                    12217 non-null  int64  
 1   Host Response Rate            12217 non-null  float64
 2   Accomodates                   12217 non-null  float64
 3   Price                         12217 non-null  float64
 4   Guests Included               12217 non-null  float64
 5   Min Nights                    12217 non-null  float64
 6   Reviews                       12217 non-null  float64
 7   Overall Rating                12217 non-null  float64
 8   review_mnt                    12217 non-null  Float64
 9   Host_tenure                   12217 non-null  float64
 10  Is Superhost                  12217 non-null  bool   
 11  Is Exact Location             12217 non-null  bool   
 12  Instant Bookable              12217 non-null  bool   
 13  T

In [4]:
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame and it contains both the features and the target
# Define your feature set (X) and target variable (y)
X = df.drop('Price', axis=1)  # Replace 'Price' with the name of your target column
y = df['Price']  # Replace 'Price' with the name of your target column

# Lasso Regression: Lasso is a linear model that applies L1 regularization to select important features
lasso = Lasso(alpha=0.01).fit(X, y)  # Fit Lasso model with alpha as regularization strength
lasso_selected = (np.abs(lasso.coef_) > 0).astype(int)  # Select non-zero coefficients

# Gradient Boosting Regressor: This model provides feature importances based on decision trees
gb = GradientBoostingRegressor().fit(X, y)  # Fit Gradient Boosting Regressor model
gb_selected = (gb.feature_importances_ > 0).astype(int)  # Select features with positive importances

# Random Forest Regressor: This model also provides feature importances
rf = RandomForestRegressor().fit(X, y)  # Fit Random Forest model
rf_selected = (rf.feature_importances_ > 0).astype(int)  # Select features with positive importances

# Ridge Regression: Ridge is a linear model that applies L2 regularization, similar to Lasso but with no sparsity
ridge = Ridge(alpha=0.01).fit(X, y)  # Fit Ridge model with alpha as regularization strength
ridge_selected = (np.abs(ridge.coef_) > 0).astype(int)  # Select non-zero coefficients

# XGBoost Regressor: This model provides feature importances based on gradient boosting
xgb = XGBRegressor().fit(X, y)  # Fit XGBoost model
xgb_selected = (xgb.feature_importances_ > 0).astype(int)  # Select features with positive importances

# Ensure all selection arrays are the same length as the number of features
num_features = X.shape[1]

# Create a DataFrame to store results
# The columns indicate if a feature is selected (1) or not (0) by each model
selection_df = pd.DataFrame({
    'Feature': X.columns,  # Column names for the features
    'Lasso': lasso_selected,  # Selection results from Lasso
    'GradientBoost': gb_selected,  # Selection results from Gradient Boosting
    'RandomForest': rf_selected,  # Selection results from Random Forest
    'Ridge': ridge_selected,  # Selection results from Ridge
    'XGBoost': xgb_selected  # Selection results from XGBoost
})

# Sum the number of selections for each feature across all models
selection_df['Sum'] = selection_df[['Lasso', 'GradientBoost', 'RandomForest', 'Ridge', 'XGBoost']].sum(axis=1)

# Output the results
# The DataFrame now includes the selection result of each feature and the total count of selections


C:\Users\AB06K\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.14013e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [5]:
selection_df

,Feature,Lasso,GradientBoost,RandomForest,Ridge,XGBoost,Sum
0,Listing ID,1,1,1,1,1,5
1,Host Response Rate,1,1,1,1,1,5
2,Accomodates,1,1,1,1,1,5
3,Guests Included,1,1,1,1,1,5
4,Min Nights,1,1,1,1,1,5
5,Reviews,1,1,1,1,1,5
6,Overall Rating,1,1,1,1,1,5
7,review_mnt,1,1,1,1,1,5
8,Host_tenure,1,1,1,1,1,5
9,Is Superhost,1,1,1,1,1,5


In [6]:
# Filter the selection_df to only include rows where the 'Sum' is less than 5
filtered_df = selection_df[selection_df['Sum'] < 5]

# Output the filtered DataFrame
filtered_df

,Feature,Lasso,GradientBoost,RandomForest,Ridge,XGBoost,Sum
10,Is Exact Location,1,0,1,1,1,4
15,Sentiment_Neg_per,0,1,1,1,1,4
17,Accuracy Rating_encoded,1,0,1,1,1,4
19,Checkin Rating_encoded,1,0,1,1,1,4


In [7]:
df = df.drop(['Is Exact Location','Accuracy Rating_encoded','Checkin Rating_encoded','Sentiment_Neg_per'], axis=1)

In [8]:
df.to_pickle(r'\\popnasal07.bnhpgroup.com\Departments\DIGITALFACTORY\GENERAL\קורס DS - AIRBNB\f_selection.pkl') 